In [7]:
import gradio as gr
import re
import subprocess
import os
import glob
import json
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from _config_ import _config_

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["OPENAI_API_KEY"] = "sk-8F9n3GqEgKlV45Js7fE8Bf3285Bc47A6961035F272F3D256"
os.environ["OPENAI_API_BASE"] = "https://api.aiwaves.cn/v1"


llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


loader = JSONLoader(
    # file_path=latest_file,
    file_path=_config_.dev_json_file,
    jq_schema='.[].desc',
    text_content=False)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [9]:
# Contextualize question ###
contextualize_q_system_prompt = (
    "根据对话历史和用户最新提出的问题，"
    "这个问题可能涉及对话历史中的上下文。"
    "请重新构造一个独立的、不依赖于对话历史即可理解的问题。"
    "如果需要，请重新表述问题；如果不需要，就原样返回。"
    "不要回答问题。"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
system_prompt = (
    "你是一个问答任务的助手。"
    "使用以下检索到的上下文(context)片段来回答这个问题。"
    "如果你不知道答案，就说你不知道。"
    "\n\n"
    "{context}"
)

# prompt = hub.pull("rlm/rag-prompt")
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )



query = "请生成一个关于重庆2天2000元预算旅游的简单描述脚本"
# query = f"请生成一个关于{destination}{days}天{budget}元预算旅游的{detail_level}描述脚本，包括具体开支情况"

# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [10]:

### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# response = rag_chain.invoke({"input": f"{query}"})
# response["answer"]
conversational_rag_chain.invoke(
    {"input": query},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


'假设我们有2000元的预算，我们可以在重庆度过两天的旅行。首日，我们可以先去解放碑，这是重庆的地标级建筑，然后到八一好吃街品尝各式美食。下午，我们可以乘坐两江小渡，感受复古的渡轮，其费用仅为10元。随后，我们可以去龙门浩老街感受另一种风格的街道。当晚，我们可以在洪崖洞欣赏夜景，感受“千与千寻”的即视感。\n\n第二天，我们可以先去解放碑，然后参观鹅岭二厂和李子坝，感受重庆轻轨穿楼的独特景象。午餐我们可以在马房湾66号江湖菜或山城二十二川菜馆品尝地道的重庆菜。下午，我们可以去涂鸦一条街和磁器口，体验重庆的文化气息。晚上，我们可以在南山品尝烧烤，尝试沈姐烤鱼等特色美食。\n\n这个旅行计划主要集中在市区内，所以交通费用不会太高。餐饮预算可以根据个人喜好调整，但总体来说2000元的预算足以在重庆度过两天愉快的旅行。'

In [11]:
query = "我不想去解放碑，换一个景点"
conversational_rag_chain.invoke(
    {"input": query},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'当然可以。如果你不想去解放碑，我们可以将它替换成鹅岭公园。鹅岭公园位于重庆市区，是一个可以欣赏到重庆全景的好地方。\n\n所以，首日我们可以先去鹅岭公园欣赏城市景色，然后到八一好吃街品尝各式美食。下午，我们可以乘坐两江小渡，感受复古的渡轮，其费用仅为10元。随后，我们可以去龙门浩老街感受另一种风格的街道。当晚，我们可以在洪崖洞欣赏夜景，感受“千与千寻”的即视感。\n\n第二天，我们可以先参观鹅岭二厂和李子坝，感受重庆轻轨穿楼的独特景象。午餐我们可以在马房湾66号江湖菜或山城二十二川菜馆品尝地道的重庆菜。下午，我们可以去涂鸦一条街和磁器口，体验重庆的文化气息。晚上，我们可以在南山品尝烧烤，尝试沈姐烤鱼等特色美食。\n\n这个旅行计划主要集中在市区内，所以交通费用不会太高。餐饮预算可以根据个人喜好调整，但总体来说2000元的预算足以在重庆度过两天愉快的旅行。'

In [27]:
query = "我也不想去鹅岭公园，换一个景点"
result = []
for s in conversational_rag_chain.stream(
    {"input": query},
    config={"configurable": {"session_id": "abc123"}},
):
    result.append(s)
    if 'answer' in s:
        print(s['answer'], end='', flush=True)

了解您的需求，我们可以将鹅岭公园替换为重庆的其他景点。考虑到您已经排除了解放碑和鹅岭公园，我们可以选择前往南山，它是重庆一个非常著名的观景点，提供了城市的全景视角。

### 两天重庆旅行计划（不含解放碑和鹅岭公园）

**第一天:**

- **上午:** 从您的住处出发，前往南山一棵树观景台，那里可以一览重庆市区和两江的壮丽景色。这里是摄影爱好者和风景欣赏者的理想之地。
- **中午:** 在南山附近的餐厅享用当地特色午餐。
- **下午:** 前往洪崖洞，这是一个将传统建筑与现代商业完美结合的景区，逛逛特色商店，体验当地文化。
- **晚上:** 在南滨路沿江散步，这里有许多美食和酒吧，可以在这里享受重庆的夜生活和夜景。

**第二天:**

- **上午:** 参观磁器口古镇，这是一个保存完好的传统古镇，可以品尝到各种重庆小吃，购买一些手工艺品和纪念品。
- **中午:** 在磁器口古镇品尝当地的午餐，享受地道的重庆风味。
- **下午:** 前往长江索道，体验从高空中俯瞰两江交汇的壮丽景色，这也是重庆非常有特色的交通方式。
- **晚上:** 结束行程前，可以选择在江边的某个景观餐厅内享用晚餐，或者再次探访洪崖洞，体验不同时间段的美丽景致。

这个行程规划了重庆的自然观景点、历史古镇、文化体验和当地美食。2000元预算应该足够覆盖两天内的食宿、交通和门票等基本开销。希望这个计划能帮助您安排一次愉快的重庆之旅！

In [25]:
result

[{'input': '我也不想去鹅岭公园，换一个景点'},
 {'chat_history': [HumanMessage(content='请生成一个关于重庆2天2000元预算旅游的简单描述脚本'),
   AIMessage(content='假设我们有2000元的预算，我们可以在重庆度过两天的旅行。首日，我们可以先去解放碑，这是重庆的地标级建筑，然后到八一好吃街品尝各式美食。下午，我们可以乘坐两江小渡，感受复古的渡轮，其费用仅为10元。随后，我们可以去龙门浩老街感受另一种风格的街道。当晚，我们可以在洪崖洞欣赏夜景，感受“千与千寻”的即视感。\n\n第二天，我们可以先去解放碑，然后参观鹅岭二厂和李子坝，感受重庆轻轨穿楼的独特景象。午餐我们可以在马房湾66号江湖菜或山城二十二川菜馆品尝地道的重庆菜。下午，我们可以去涂鸦一条街和磁器口，体验重庆的文化气息。晚上，我们可以在南山品尝烧烤，尝试沈姐烤鱼等特色美食。\n\n这个旅行计划主要集中在市区内，所以交通费用不会太高。餐饮预算可以根据个人喜好调整，但总体来说2000元的预算足以在重庆度过两天愉快的旅行。'),
   HumanMessage(content='我不想去解放碑，换一个景点'),
   AIMessage(content='当然可以。如果你不想去解放碑，我们可以将它替换成鹅岭公园。鹅岭公园位于重庆市区，是一个可以欣赏到重庆全景的好地方。\n\n所以，首日我们可以先去鹅岭公园欣赏城市景色，然后到八一好吃街品尝各式美食。下午，我们可以乘坐两江小渡，感受复古的渡轮，其费用仅为10元。随后，我们可以去龙门浩老街感受另一种风格的街道。当晚，我们可以在洪崖洞欣赏夜景，感受“千与千寻”的即视感。\n\n第二天，我们可以先参观鹅岭二厂和李子坝，感受重庆轻轨穿楼的独特景象。午餐我们可以在马房湾66号江湖菜或山城二十二川菜馆品尝地道的重庆菜。下午，我们可以去涂鸦一条街和磁器口，体验重庆的文化气息。晚上，我们可以在南山品尝烧烤，尝试沈姐烤鱼等特色美食。\n\n这个旅行计划主要集中在市区内，所以交通费用不会太高。餐饮预算可以根据个人喜好调整，但总体来说2000元的预算足以在重庆度过两天愉快的旅行。'),
   HumanMessage(content='我也不想去鹅岭公园，换一个景点'),
   AIMessage(conte